# Milestone Project 1: Food Vision

* Use TensorFlow datasets to download and explore data
* Create a preprocessing function for the data
* Batch and prepare datasets for modelling
* Set up mixed precision training

Food101
    Training: 75,750 images (750 per class)
    Testing: 250 images per class

In [ ]:
!nvidia-smi
# Need compute capability score of 7.0 or higher for mixed precision training
# Mine = GeForce RTX 3070 Notebook GPU 
#   Compute capability: 8.6


In [ ]:
# Get custom images
!wget -nc -P ../Downloads/ https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py


In [ ]:
# Import series of helper functions for the notebook
import sys
# append the downloads path
sys.path.append("../Downloads")
from _helper_functions import create_tensorboard_callback, plot_loss_curves, compare_historys

# Use TensorFlow datasets to download and explore data

In [ ]:
# TFDS - Tensorflow Datasets
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
# List datasets
datasets_list = tfds.list_builders()
print("food101" in datasets_list)

In [ ]:
# Check download size (some datasets are HUGE!)
food_101_info = tfds.builder(name='food101').info
print(food_101_info.download_size)

In [ ]:
(train_data, test_data), ds_info = tfds.load(name="food101",
                                            data_dir="../Downloads",
                                            split=["train", "validation"],
                                            shuffle_files=True,
                                            as_supervised=True, # data returned as tuple (data, label)
                                            with_info=True)

## Exploring Food101 Data from TensorFlow Datasets

* Class names
* Shape of input data (image tensors)
* Datatype of input data
* One-hot encoded or label encoded
* Do the labels match up with the class names?

In [ ]:
# Features of Food101 from TFDS
print(ds_info.features)

In [ ]:
# Get class names
class_names = ds_info.features["label"].names
class_names[:10]

In [ ]:
# Take one sample from the train dataset

train_one_sample = train_data.take(1) # samples are in format (image_tensor, label)
train_one_sample

In [ ]:
# Output info about the training sample
for image, label in train_one_sample:
    print(f"""
    Image shape: {image.shape}
    Image datatype: {image.dtype}
    Target class from Food101 (tensor form): {label}
    Class name (str form): {class_names[label.numpy()]}
    """)

In [ ]:
# What does the image tensor from TFDS's Food101 look like?
image

In [ ]:
# Check min and max values of the image tensor
tf.reduce_min(image), tf.reduce_max(image)

### Plot Image

In [ ]:
# Plot image tensor
import matplotlib.pyplot as plt
plt.imshow(image)
plt.title(class_names[label.numpy()])
plt.axis(False)
plt.show()


## Create Preprocessing Functions for the Data

Neural networks perform best when data is:
* Batched
* Normalized
* etc.

The Data:
* Datatype - `uint8` dtype
* Shape  - different sized images
* Not Scaled (pixel values are betwween 0-255)

What we need:
* Data in `float32` dtype
* For batches, TensorFlow likes all the tensors within a batch to be of the same size.
* Scaled (normalized) values between 0-1




In [ ]:
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
def preprocess_img(image, label, img_shape=224, normalize=False):
    """
    Converts image datatype to `float32` and reshapes image to [img_shape, img_shape, color_channels]
    """
    image = tf.image.resize(image, [img_shape, img_shape])
    image = tf.cast(image, tf.float32)
    if normalize:
        image = image / 255.0
    return image, label

In [ ]:
# Preprocess a single sample image and check outputs
preprocessed_img = preprocess_img(image, label, normalize=False)[0]
print(f"Image before preprocessing:\n {image[:2]}...,\nShape:{image.shape},\nDatatype:{image.dtype}")
print(f"Image after prerpocessing:\n {preprocessed_img[:2]}...,\nShape:{preprocessed_img.shape},\nDatatype:{preprocessed_img.dtype}")

## Batch and Prepare Datasets for Modeling

Make data input pipeline run really fast.

Reading: https://www.tensorflow.org/guide/data

`image_dataset_from_directory`

In [ ]:
# Map preprocessing function to training (parallelize)
train_data = train_data.map(map_func=preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)
# Shuffle train_data, turn it into batches and prefetch it (load it faster)
train_data = train_data.shuffle(buffer_size=1024).batch(batch_size=128).prefetch(buffer_size=tf.data.AUTOTUNE)

# Map preprocessing function to test data
test_data = test_data.map(map_func=preprocess_img, num_parallel_calls=tf.data.AUTOTUNE).batch(batch_size=128).prefetch(buffer_size=tf.data.AUTOTUNE).cache()

### Breakdown

1. Map the preprocessing function (`preprocess_image`) across the training dataset
2. Shuffle the dataset using buffer size of 1024
3. Batch the dataset using a batch size of 32
4. Prefetch the next batch (using `tf.data.experimental.AUTOTUNE`)

## Create Modelling Callbacks

* TensorBoard callback to log training results.
* ModelCheckpoint callback to save our model's progress after feature extraction

In [ ]:
# Create tensorboard callback (import from _helper_functions.py)
from _helper_functions import create_tensorboard_callback

# Create ModelCheckpoint callback to save a model's progress after each epoch
checkpoint_path = "../checkpoints/food_101_milestone/milestone_1.ckpt"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                    monitor="val_accuracy", 
                                                    save_best_only=True,                                                     
                                                    save_weights_only=True,
                                                    verbose=0)

## Set Up Mixed Precision Training

[TensorFlow Mixed Precision](https://www.tensorflow.org/guide/mixed_precision)

Mixed precision is the use of both 16-bit and 32-bit floating-point types in a model during training to make it run faster and use less memory. 



In [ ]:
# Turn on mixed precision training
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import mixed_precision

print(tf.__version__)

In [ ]:
tf.keras.mixed_precision.set_global_policy('mixed_float16')
mixed_precision.global_policy()

## Build Feature Extraction Model


In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
mixed_precision.global_policy()
# Create base model
input_shape = (224, 224, 3)
base_model = tf.keras.applications.EfficientNetV2B0(include_top=False)
base_model.trainable = False # freeze base model layers

# Create Functional model 
inputs = layers.Input(shape=input_shape, name="input_layer")
# Note: EfficientNetBX models have rescaling built-in but if your model didn't you could have a layer like below
# x = preprocessing.Rescaling(1./255)(x)
x = base_model(inputs, training=False) # set base_model to inference mode only
x = layers.GlobalAveragePooling2D(name="pooling_layer")(x)
x = layers.Dense(len(class_names))(x) # want one output neuron per class 
# Separate activation of output layer so we can output float32 activations
outputs = layers.Activation("softmax", dtype=tf.float32, name="softmax_float32")(x) 
model = tf.keras.Model(inputs, outputs)

# Compile the model
model.compile(loss="sparse_categorical_crossentropy", # Use sparse_categorical_crossentropy when labels are *not* one-hot
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

In [ ]:
for layer in model.layers:
    print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

In [ ]:
# check base model layers
for layer in base_model.layers:
    print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

## Fit the feature extraction model

1. build a feature extraction model (train a couple output layers with base layers frozen)
2. Fine-tune some of the frozen layers

In [ ]:
history_101_food_classes_feature_extract = model.fit(
    train_data, 
    epochs=3, 
    steps_per_epoch=len(train_data),
    validation_data=test_data, 
    validation_steps=int(0.15 * len(test_data)),
    callbacks=[
        create_tensorboard_callback(
            dir_name="../training_logs", 
            experiment_name="efficientnetv2b0_101_classes_all_data_feature_extract"
        ),
        model_checkpoint
    ]
)

In [ ]:
# Evaluate on whole dataset
results_feature_extract_model = model.evaluate(test_data)
results_feature_extract_model


## Fine Tune

Fine tune the model to beat the top-1 accuracy of 77.4%

### Load and Evaluate Checkpoint Weights
We can load in and evaluate our model's checkpoints by:

1. Cloning our model using `tf.keras.models.clone_model()` to make a copy of our feature extraction model with reset weights.
2. Calling the `load_weights()` method on our cloned model passing it the path to where our checkpointed weights are stored.
3. Calling `evaluate()` on the cloned model with loaded weights.

In [ ]:
# Clone the model we created (this resets all weights)
cloned_model = tf.keras.models.clone_model(model)
cloned_model.summary()

In [ ]:
# Where are our checkpoints stored?
checkpoint_path

In [ ]:
# Load checkpointed weights into cloned_model
cloned_model.load_weights(checkpoint_path)

In [ ]:
# Compile cloned_model (with same parameters as original model)
cloned_model.compile(loss="sparse_categorical_crossentropy",
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=["accuracy"])

In [ ]:
# Evalaute cloned model with loaded weights (should be same score as trained model)
results_cloned_model_with_loaded_weights = cloned_model.evaluate(test_data)

In [ ]:
# Loaded checkpoint weights should return very similar results to checkpoint weights prior to saving
import numpy as np
# check if all elements in array are close
np.isclose(results_feature_extract_model, results_cloned_model_with_loaded_weights).all() 


In [ ]:
# Check the layers in the base model and see what dtype policy they're using
for layer in cloned_model.layers[1].layers[:20]: # check only the first 20 layers to save space
  print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

In [ ]:
# Save model locally
save_dir = "../models/07_efficientnetb0_feature_extract_model_mixed_precision"
model.save(save_dir)

### Load from Here!

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from _helper_functions import create_tensorboard_callback

# Load model previously saved above
save_dir = "../models/07_efficientnetb0_feature_extract_model_mixed_precision"
loaded_saved_model = tf.keras.models.load_model(save_dir)
(train_data, test_data), ds_info = tfds.load(name="food101",
                                            data_dir="../Downloads",
                                            split=["train", "validation"],
                                            shuffle_files=True,
                                            as_supervised=True, # data returned as tuple (data, label)
                                            with_info=True)
                                            # Map preprocessing function to training (parallelize)

def preprocess_img(image, label, img_shape=224, normalize=False):
    """
    Converts image datatype to `float32` and reshapes image to [img_shape, img_shape, color_channels]
    """
    image = tf.image.resize(image, [img_shape, img_shape])
    image = tf.cast(image, tf.float32)
    if normalize:
        image = image / 255.0
    return image, label
                                            
train_data = train_data.map(map_func=preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)
# Shuffle train_data, turn it into batches and prefetch it (load it faster)
train_data = train_data.shuffle(buffer_size=1024).batch(batch_size=128).prefetch(buffer_size=tf.data.AUTOTUNE)

# Map preprocessing function to test data
test_data = test_data.map(map_func=preprocess_img, num_parallel_calls=tf.data.AUTOTUNE).batch(batch_size=128).prefetch(buffer_size=tf.data.AUTOTUNE).cache()

In [ ]:
# Check the layers in the base model and see what dtype policy they're using
for layer in loaded_saved_model.layers[1].layers[:20]: # check only the first 20 layers to save output space
  print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

In [ ]:
# Check loaded model performance (this should be the same as results_feature_extract_model)
# results_loaded_saved_model = loaded_saved_model.evaluate(test_data)
# results_loaded_saved_model

In [ ]:
# The loaded model's results should equal (or at least be very close) to the model's results prior to saving
# Note: this will only work if you've instatiated results variables 
import numpy as np
# Evaluate on whole dataset
#np.isclose(results_feature_extract_model, results_loaded_saved_model).all()

### Prepare the model's layers for fine-tuning

In [ ]:
loaded_saved_model.summary()

In [ ]:
# make all layers trainable/unfreeze
for layer in loaded_saved_model.layers:
  layer.trainable = True # set all layers to trainable
  print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy) # make sure loaded model is using mixed precision dtype_policy ("mixed_float16")

In [ ]:
# Check the layers in the base model and see what dtype policy they're using
for layer in loaded_saved_model.layers[1].layers[:20]:
  print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

### Early Stopping Callback and Model Checkpoint Callback

In [ ]:
# Setup EarlyStopping callback to stop training if model's val_loss doesn't improve for 3 epochs
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", # watch the val loss metric
                                                  patience=3) # if val loss decreases for 3 epochs in a row, stop training

# Create ModelCheckpoint callback to save best model during fine-tuning
checkpoint_path = "../checkpoints/food_101_milestone/fine_tune_checkpoints.ckpt"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                      save_best_only=True,
                                                      monitor="val_loss")

### Learning Rate Reduction Callback

In [ ]:
# Creating learning rate reduction callback
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",  
                                                 factor=0.2, # multiply the learning rate by 0.2 (reduce by 5x)
                                                 patience=2,
                                                 verbose=1, # print out when learning rate goes down 
                                                 min_lr=1e-7)

In [ ]:
# Compile the model
loaded_saved_model.compile(loss="sparse_categorical_crossentropy", # sparse_categorical_crossentropy for labels that are *not* one-hot
                        optimizer=tf.keras.optimizers.Adam(0.0001), # 10x lower learning rate than the default
                        metrics=["accuracy"])

In [ ]:
# Start to fine-tune (all layers)
history_101_food_classes_all_data_fine_tune = loaded_saved_model.fit(train_data,
                                                        epochs=100, # fine-tune for a maximum of 100 epochs
                                                        steps_per_epoch=len(train_data),
                                                        validation_data=test_data,
                                                        validation_steps=int(0.15 * len(test_data)), # validation during training on 15% of test data
                                                        callbacks=[
                                                            create_tensorboard_callback(
                                                                dir_name="../training_logs", 
                                                                experiment_name="efficientv2b0_101_classes_all_data_fine_tuning",
                                                            ), # track the model training logs
                                                            model_checkpoint, # save only the best model during training
                                                            early_stopping, # stop model after X epochs of no improvements
                                                            reduce_lr
                                                            ]
                                                        ) # reduce the learning rate after X epochs of no improvements

In [ ]:
loaded_saved_model.evaluate(test_data)